__Вопросы для повторения:__

* Что выведет программа?

    ```c++
    std::string name = "Ilya";
    std::string surname = "Muromec";

    std::string& r = name;
    r = "Dobrynia";
    std::cout << name << std::endl;

    std::string* p = &surname;
    surname = "Nikitich";
    std::cout << p << std::endl;
    ```

* Что следует сказать про эту программу?

```c++
std::vector<int> gen_sequence()
{
    return {1, 2, 3, 4, 5};
}

const int * search_value(const std::vector<int>& v, int value)
{
    for (const int& x : v)
        if (x == value)
            return &x;
    return nullptr;
}

if (const int* val = search_value(gen_sequence(), 5))
    std::cout << *val;
```

* Каковы контракты функций. Какие юнит-тесты следует написать?

```c++
void sort(int* arr, int n);

void sort(std::vector<int>& v);

int to_int(const std::string& s);
```

<br />

##### Эхо прошлой лекции: пространства имён и name mangling при компиляции

https://en.wikipedia.org/wiki/Name_mangling

Вы уже несколько раз втречались с `std::string`, `std::cout`, `std::vector`. `std` - пространство имён с функциями и классами стандартной билиотеки С++.

<br />

Пространства времён решают проблему, когда функция с одинаковым именем имеет две реализации в разных билиотеках

```c++
// library for json parsing
//
// return version of json parsing library
std::string get_version();

// library for image manipulation
//
// return version of image manipulation library
std::string get_version();
```

Такой код скомпилируется, но не слинкуется (либо слинкуется в некорректную программу), т.к. для функции `get_version` нарушено ODR-правило (One Definition Rule)

```c++
namespace json_parser
{
    // return version of json parsing library
    std::string get_version();
}

namespace image_processing
{
   // return version of image manipulation library
    std::string get_version(); 
}
```

Такой код слинкуется корректно, т.к. работает механизм name mangling во время компиляции

Показать пример на godbolt.org, не забыть убрать галочку demangle

```c++
#include <iostream>
#include <string>

std::string get_version(int x, int y, long z);

namespace json_parser
{
    std::string get_version(int x, int y, long z);
}

namespace image_processing
{
    std::string get_version(int x, int y, long z);
}

int main()
{
    std::cout << get_version(0, 0, 0);
    std::cout << json_parser::get_version(0, 0, 0);
    std::cout << image_processing::get_version(0, 0, 0);
    return 0;
}
```

В результате name mangling компилятор clang создаёт символы:
* `_Z11get_versionB5cxx11iil`
* `_ZN16json_parser11get_versionB5cxx11Eiil`
* `_ZN21image_processing11get_versionB5cxx11Eiil`

Какой из них к чему относится, догадаться не сложно. 

<br />

##### Поиск функции и `using namespace`

Точные правила поиска:

https://en.cppreference.com/w/cpp/language/lookup

При компиляции кода:

```c++
namespace json_parser {
namespace input_processing {

int read_int(const std::string& s)
{
    if (avx_instructions_available())
        return read_int_avx(s);
    else
        return read_int_default(s);
}
    
}
}
```

Компилятору нужно найти кандидатов для вызова `avx_instructions_available`, `read_int_avx` и `read_int_default`. Компилятор осуществляет их поиск в пространствах имён:
* глобальное
* `json_parser`
* `json_parser::input_processing`
* пространства имён аргументов функций (ничего для `avx_instructions_available`, `std` для `read_int_avx` и `read_int_default`)

Если для какой-нибудь из функций находится либо более одного кандидата для вызова либо ни одного - ошибка компиляции. Кандидат должен быть и быть только один.

<br />

`using namespace X` - заполнить текущее пространство имён до закрываеющей скобки именами из пространства X.

**Пример 1:**

Вызывающая программа:

```c++
#include "json_parser.h"

int main()
{
    // не скомпилируется, т.к. в глобальном пространстве имён нет функции read_int
    std::cout << read_int(s);
    
    // скомпилируется
    std::cout << json_parser::input_processing::read_int(s);
    
    // скомпилируется, т.к. глобальное пространство имён расширено
    using namespace json_parser::input_processing;
    std::cout << read_int(s);
    
    return 0;    
}
```

**Пример 2:**

```c++
int main() {
    std::cout << 1 << std::string("23");
    return 0;
}
```

или

```c++
using namespace std; 
// имена из std доступны в глобальном пространстве имён до конца файла

int main() {
    cout << 1 << string("23");
    return 0;
}
```

или

```c++
int main() {
    using namespace std; 
    // имена из std доступны в глобальном пространстве имён до конца main
    
    cout << 1 << string("23");
    return 0;
}
```

<br />

**Рекомендации:**

1. Никогда не пишите `using namespace` в `.h` - файлах (почему?)
2. Ограничивайте область действия `using namespace` рационально.

**Пример рациональности:** функция, считающая время выполнения другой функции (многословность `std::chrono`):

```c++
// без using namespace
std::uint64_t get_execution_time_microseconds(std::function<void> f)
{
    const std::chrono::high_resolution_clock::time_point start_ts =
        std::chrono::high_resolution_clock::now();
    f();    
    const std::chrono::high_resolution_clock::time_point final_ts =
        std::chrono::high_resolution_clock::now();    
    return std::chrono::duration_cast<std::chrono::microseconds>(final_ts - start_ts).count();
}

// с using namespace
std::uint64_t get_execution_time_microseconds(std::function<void> f)
{
    using namespace std::chrono;    
    const high_resolution_clock::time_point start_ts = high_resolution_clock::now();
    f();    
    const high_resolution_clock::time_point final_ts = high_resolution_clock::now();    
    return duration_cast<microseconds>(final_ts - start_ts).count();
}
```

<br />

### Лекция 3. Классы. Введение

Класс и объект - ключевые термины в ООП - языках

![](classes.jpg)

* Барсик - объект класса "Кошка домашняя"
* "Кошка домашняя" - класс, наследник класса "Кошки"
* "Кошки" - класс, наследник класса "Кошачьи"
* "Кошачьи" - класс, наследник класса "Хищные"


Объект класса - конкретный элемент этого типа.

Класс-наследник __уточняет__ характеристики базового класса

Для описания классов в языке С++ есть два ключевых слова: `class` и `struct`

```c++
class Animal
{
    ...
};

struct Point
{
    ...
};
```

Класс может иметь данные: члены класса

```c++
class Animal
{
    std::string name;
    unsigned age;
};

struct Point
{
    float x;
    float y;
};
```

Как создавать объекты таких классов:

```c++
Animal a;
Point p;
```

Доступ до полей:

```c++
p.x = 3.f;
p.y = 4.f;
```

Класс может иметь методы:

```c++
class Animal
{
    bool is_too_young() const
    {
        return age == 0;
    }
   
    std::string name;
    unsigned age;
};

//
// или так:
//

class Animal
{
    bool is_too_young() const;
    
    std::string name;
    unsigned age;
};

bool Animal::is_too_young() const
{
    return age == 0;
}
```

Класс может наследоваться от другого класса:

```c++
class Cat : Animal
{
};

struct WeightedPoint : Point
{
    float w;
};
```

<br />

У членов и методов классов есть модификаторы видимости:
* `private` - доступ только себе самому и друзьям
* `protected` - плюс наследники
* `public` - кто угодно

Отличие `class` от `struct` (с точки зрения компилятора) - модификатор видимости по умолчанию для методов/членов и при наследовании.
* `class` -> `private`
* `struct` -> `public`

```c++
class Animal
{
public:
    bool is_too_young() const { return age == 0; }
  
private:
    std::string name;
    unsigned age;
};

class Cat : public Animal
{
public:
    void say_meow() const { std::cout << "meow"; }
};

struct Point
{
    float x;
    float y;
};

struct WeightedPoint : Point
{
    float w;
};
```

```c++
Cat barsic;
barsic.say_meow();      // OK
barsic.name = "Barsic"; // Compile-time error: name is private member
    
WeightedPoint p;
p.x = 3.f;  // OK
p.w = 0.f;  // OK
```

<br />

Иногда приходится создавать объекты классов, копировать и уничтожать их. Для этого существуют конструкторы и деструкторы:

```c++
class Animal
{
private:
    std::string name;
    unsigned age;
    
public:
    // конструктор из двух аргументов
    //
    // Animal an("Murka", 7);
    Animal(const std::string& a_name, const int a_age)
        // список инициализации для членов класса
        // плюс, возможно, спецификация вызова базового класса
        : name(a_name)
        , age(a_age)
    {
        // тело конструктора
        std::cout << "creating animal" << std::endl;
        std::cout << "animal created" << std::endl;
    }  // когда выполнение доходит до конца конструктора, объект класса Animal считается сконструированным
    
    // конструктор из одного аргумента
    //
    // Animal an("Murka");
    explicit Animal(const std::string& a_name)
        : name(a_name)
        , age(0)
    {
    }
    
    // конструктор по умолчанию
    //
    // Animal an;
    Animal() : age(0) {}
    // что с name?
    
    // конструктор копирования
    //
    // Animal a1;  // default
    // Animal a2 = a1;
    // Animal a3(a2);
    Animal(const Animal& rhs)
        // имеет доступ к приватным полям rhs, т.к. объект того же класса
        : name(rhs.name), age(rhs.age)
    {}
    
    // конструктор перемещения
    //
    // Animal a1("Murka", 13);
    // Animal a2 = std::move(a1);
    // a2 - ? a1 - ?
    Animal(Animal&& rhs) noexcept : name(std::move(rhs.name)), age(rhs.age) {}
    
    // деструктор
    // TODO: обратить внимание, что в деструкторе происходит больше операций, чем вывод,
    //       объяснить, почему.
    ~Animal() { std::cout << "destroying " << name << std::endl; }
    
    // копирующее присваивание
    //
    // Animal a1("Murka", 13), a2;
    // a2 = a1;
    Animal& operator = (const Animal& rhs) { ...; return *this; }
    
    // перемещающее присваивание
    //
    // Animal a1("Murka", 13), a2;
    // a2 = std::move(a1);
    Animal& operator = (Animal&& rhs) noexcept { ...; return *this; }
};
```

Для каждого класса могут быть определены (или удалены) основные операции:
* default c-tor
* copy c-tor
* move c-tor
* copy assign
* move assign
* d-tor

<br />

##### Когда вызываются конструкторы и деструкторы

Конструктор класса вызывается в момент его создания. Пока конструктор класса не отработает, объектом пользоваться нельзя.

Деструктор класса вызывается, когда объект покидает свою область видимости.

```c++
void fun()
{
    Grasshopper barsic1("Barsic", 1);
    
    {
        Grasshopper barsic2("Barsic", 2);
    }
    
    Grasshopper barsic3("Barsic", 3);
}
```

```c++
void fun()
{
    Grasshopper* barsic = new Grasshopper("Barsic", 1);
    
    for (int i = 0; i < 10; ++i)
    {
        Grasshopper barsic2("Barsic", i);
    }
}
// Обсудить:
//    1. последовательность конструкторов-деструкторов
//    2. какая проблема
//    3. как поправить
```

```c++
void fun()
{
    Grasshopper* barsics = new Grasshopper[3];
    
    if (Grasshopper h = make_grasshopper(); h.is_alive())
    {
        std::cout << "it is alive!" << std::endl;
    }
    else
    {
        std::cout << "" << std::endl;
    }
    
    delete barsics;
}
// Обсудить:
//    1. последовательность конструкторов-деструкторов
//    2. какая проблема
//    3. как поправить
```

<br />

##### RAII (Resource Acqusition is Initialization)

RAII - ключевая идиома в С++: захват ресурса должен совпадать с инициализацией объекта, а освобождение ресурса с его разрушением.

Т.о.:
* конструктор объекта захватывает ресурс
* деструктор объекта его освобождает

**Пример 1**: массив на куче

```c++
class ArrayD
{
private:
    double* data;
    int size;
    
public:
    ArrayD(int n)
        : data(new double[n]),
        , size(n)
    {
    }
    
    ~ArrayD()
    {
        delete[] data;
    }    
};

void func()
{
    ArrayD arr(100);    
    ...
}
```

Теперь вы можете представить как работает такой код с памятью и почему он корректен:

```c++
void fun()
{
    std::vector<std::string> team = {"Dobrynia", "Ilusha", "Alesha"};
    
    for (const auto& member : team)
        std::cout << member << std::end;    
}
```

**Пример 2**: С++-обёртка над С-файлом

```c++
class File
{
private:
    FILE* file;
public:
    File(const char* filename) : file(std::fopen(filename, "w+"))
    {
        if(!file)
            throw std::runtime_error("failed to open file") ;
    }
    
    ~File()
    {
        std::fclose(file);
    }
    ...
};


void fun()
{
    File f("input.txt");
    ...
}
```

<br />

##### Какую часть работы по составлению конструкторов, деструкторов и операторов присваивания можно отдать компилятору?

Компилятор может сгенерировать эти операции в дефолтном виде **по полям**, правила такие:

Что имеется ввиду под фразой **по полям** :

```c++
class Animal
{
    std::string name = "UNK";
    int age = 0;
};

Animal an1;           // значение?
Animal an2(an1);      // значение?
an1 = an2;            // значение?
an2 = std::move(an1); // значение?
```

* автогенерируемый дефолтный конструктор устанавливает полям дефолтные значения
* автогенерируемый конструктор копирования копирует значения по полям
* автогенерируемый конструктор перемещения перемещает значения по полям
* автогенерируемый оператор присваивания ... (вопрос к студентам)
* автогенерируемый оператор перемещения ... (вопрос к студентам)
* автогенерируемый деструктор ... (вопрос к студентам)

__Вопрос__: В каких случаях нужно избежать автогенерированного конструктора копирования? Пример?

<details>
<summary>Ответ 1</summary>
<p>

Когда копирование по полям некорректно копирует состояние класса. Пример:

```c++
class String
{
    const char* data;
    unsigned int len;
};
```

</p>
</details>

<details>
<summary>Ответ 2</summary>
<p>

Когда копирование запрещено. Пример:

```c++
class File
{
    FILE* file;
};
```

</p>
</details>

__Вопрос для продвинутых__: В каких ещё случаях (кроме ручного deepcopy) нужно избежать автогенерированного конструктора перемещения? Пример?

<details>
<summary>Ответ</summary>
<p>

Когда есть зависимость по полям. Пример:


```c++
class ListWithZerosCount
{
    std::list<int> list;
    int zeros_count;
};


ListWithZerosCount l1 = ...;

ListWithZerosCount l2 = std::move(l1);

// можем продолжить работу с l1
```

Объяснение: перемещение всегда оставляет перемещаемый объект в unspecified valid state (если специально не оговорено иначе). Если по полям есть зависимость, перевод всех полей в unspecified valid state mможет нарушить корректность самого класса.

</p>
</details>


<br />

Правила автогенерации:

![](comp_rules.jpg)

Но никто их не помнит наизусть, поэтому работает правило 6:
    
__Определил одно - определи все шесть__